# Agentic AI MCP - Server Only

This notebook starts an MCP server that exposes tools.
Run this on the machine where you want to host the tools.

Another machine can connect to this server using the client notebook.

## Step 1: Setup

In [1]:
# Install if needed
# !uv pip install -e ".[dev]"

import agentic_ai_mcp
print(f"Version: {agentic_ai_mcp.__version__}")

Version: 0.4.2


## Step 2: Define Your Tools

In [2]:
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

def greet(name: str, times: int = 1) -> str:
    """Greet someone."""
    return ("Hello, " + name + "! ") * times

## Step 3: Create AgenticAI and Register Tools

Configure the host/port. Use `0.0.0.0` to allow external connections.

In [3]:
from agentic_ai_mcp import AgenticAI

# Use 0.0.0.0 to allow connections from other machines
ai = AgenticAI(
    host="0.0.0.0",  # Listen on all interfaces
    port=8888,
    verbose=True
)

# Register your tools
ai.register_tool(add)
ai.register_tool(multiply)
ai.register_tool(greet)

print(f"Registered tools: {ai.tools}")

Registered tools: ['add', 'multiply', 'greet']


## Step 4: Start the MCP Server

This will start the server in background. The server URL will be:
- Local: `http://127.0.0.1:8888/mcp`
- Remote: `http://<your-ip>:8888/mcp`

Give this URL to the client notebook.

In [4]:
# Start server in background (notebook continues running)
ai.run_mcp_server()

# Get your IP for remote connections
import socket
hostname = socket.gethostname()
try:
    local_ip = socket.gethostbyname(hostname)
except:
    local_ip = "<your-ip>"

print(f"\nServer is ready!")
print(f"Local URL:  http://127.0.0.1:8888/mcp")
print(f"Remote URL: http://{local_ip}:8888/mcp")

INFO:     Started server process [42495]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8888 (Press CTRL+C to quit)
/home/cloud/.local/share/uv/python/cpython-3.13.9-linux-x86_64-gnu/lib/python3.13/multiprocessing/popen_fork.py:67: DeprecationWarning: This process (pid=42477) is multi-threaded, use of fork() may lead to deadlocks in the child.
  self.pid = os.fork()


MCP Server running at http://0.0.0.0:8888/mcp
Tools: ['add', 'multiply', 'greet']

Server is ready!
Local URL:  http://127.0.0.1:8888/mcp
Remote URL: http://127.0.1.1:8888/mcp


## Step 5: Stop the Server

Run this cell to stop the server when you're done.

In [ ]:
ai.stop_mcp_server()